In [1]:
import nltk, re
import numpy as np
import pandas as pd

import nltk

import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

In [2]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
test = dataset.data[100:150]
articles = dataset.data

In [3]:
articles[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [4]:
corpus =[]
for doc in articles:
    d =[]
    for sent in nltk.sent_tokenize(doc):
        s =[]
        sentTag=nltk.pos_tag(nltk.word_tokenize(sent))
        for word,tag in sentTag:
            try:
                re.findall(r"[N]\w+|[V]\w+|[A-Za-z]\w+",tag)[0]
                if word not in stop_words and len(word) >1 and re.findall(r"(?:^|(?<= ))[a-zA-Z0-9]+(?= |$)",word)[0]:
                    s.append(word)
            except:
                pass
        d.append(s)
    corpus.append(d)

print(corpus[0])

[['wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day'], ['It', 'sports', 'car', 'looked', 'late', 'early', '70s'], ['It', 'called', 'Bricklin'], ['The', 'doors', 'really', 'small'], ['In', 'addition', 'front', 'bumper', 'separate', 'rest', 'body'], ['This', 'know'], ['If', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please']]


In [14]:
# for quick model check let's take only 100 rows of data
corpus = corpus[0:100]



from nltk.corpus import movie_reviews as data
corpus =[]
for fileid in data.fileids()[0:50]:
    d = []
    for sent in data.sents(fileid):
        s = []
        word_tag = nltk.pos_tag(sent)
        for word,tag in word_tag:
            try:
                re.findall(r"[N]\w+|[V]\w+",tag)[0]
                if word not in stop_words and len(word) >1 and re.findall(r"(?:^|(?<= ))[a-zA-Z0-9]+(?= |$)",word)[0]:
                    s.append(word)
            except:
                pass
        d.append(s)
    corpus.append(d)
    

In [15]:
#df = pd.read_csv("regulation.csv")
#text = df.content.values[0:7]

##corpus = []
#for doc in text:
#    corpus.append([line.split()for line in doc.split(".")])

In [16]:
vocabulary  = [word for doc in corpus for line in doc for word in line]
vocabulary =list(set(vocabulary))

In [17]:
vocabulary[0:10]

['Made',
 'minimal',
 'PostScript',
 'organized',
 'malfunction',
 'Whether',
 'Pat',
 'amps',
 'Europe',
 'opposing']

In [26]:
# indexing word 
id_word = {i:w for i,w in enumerate(vocabulary)}
word_id  = {w:i for i,w in enumerate(vocabulary)}

# corpus with word id insted of word
docs = []
for doc in corpus:
    docs.append ([[word_id[word] for word in line] for line in doc])

In [27]:
# hyper parameters:
K_gl = 5 # number of global topic 
K_loc = 5 # number of local topic

gamma         = 0.1 #parameter of sentence prior
alpha_gl      = 0.1 # parameter of global topics prior
alpha_loc     = 0.1 # parameter of local topics prior

#parameters for choosing between local and global topics
alpha_mix_gl  = 0.1
alpha_mix_loc = 0.1

beta_gl = 0.1 # parameter of global words prior
beta_loc = 0.1 # parameter of local words prior

T = 3 # sliding window width

docs = docs
W = len(vocabulary)

In [28]:

#Kgl word distributions for global topics - list(list(list(word from sent)sent from doc)doc from corpus)
v_d_s_n     = [] #sumi - 
r_d_s_n     = [] #sumi - global or local
z_d_s_n     = [] #sumi- word -topic dist

#topic distribution over words
#n prefix is for count
n_gl_z_w    = np.zeros((K_gl, W)) #zero initialized matrix of (num. global topic X no of words)
n_gl_z      = np.zeros(K_gl) # zero initialized list, len of list = number of topics.number of words in each topic
n_d_s_v     = [] #sumi
n_d_s       = [] #sumi
n_d_v_gl    = [] #sumi
n_d_v       = [] #sumi

# topic dist over doc
n_d_gl_z    = np.zeros((len(docs), K_gl)) #zero initialized matrix of (num.of doc X no globak topic)
n_d_gl      = np.zeros((len(docs))) # zero initialized list,  number of doc.

#Kloc word distributions for local topics
n_loc_z_w   = np.zeros((K_loc, W)) # matrix of size (no of local topic X vocabulary size)
n_loc_z     = np.zeros(K_loc) # number of local topic
n_d_v_loc   = [] #sumi
n_d_v_loc_z = [] #sumi

inflation = 0

print ("random fitting to initialize")
for m, doc in enumerate(docs):
    v_d = []
    r_d = []
    z_d = []

    n_d_s_v_d = []
    n_d_s_d = []
        
    n_d_v_gl_v = [inflation for v in range(T+len(doc)-1)]
    n_d_v_v = [inflation for v in range(T+len(doc)-1)]
    n_d_v_loc_v = [inflation for v in range(T+len(doc)-1)]
    n_d_v_loc_z_v  = [[inflation for k in range(K_loc)] for v in range(T+len(doc)-1)]

    n_d_v_gl.append(n_d_v_gl_v)
    n_d_v.append(n_d_v_v)
    n_d_v_loc.append(n_d_v_loc_v)
    n_d_v_loc_z.append(n_d_v_loc_z_v)
       
    for s, sent in enumerate(doc):
        v_s = [np.random.randint(0, T) for i, word in enumerate(sent)]
        r_s = ["gl" if (np.random.randint(0, 2) == 0 )else "loc" for i, word in enumerate(sent)]
        z_s = [np.random.randint(0, K_gl)if r =="gl" else np.random.randint(0, K_loc) for r in r_s]
        n_d_s_v_s = [inflation for v in range(T)]
        
        v_d.append(v_s)
        r_d.append(r_s)
        z_d.append(z_s)
        
        n_d_s_v_d.append(n_d_s_v_s)
        n_d_s_d.append(inflation) # initialize n_d_s   

    v_d_s_n.append(v_d)
    r_d_s_n.append(r_d)
    z_d_s_n.append(z_d)
    
    n_d_s_v.append(n_d_s_v_d)
    n_d_s.append(n_d_s_d)


random fitting to initialize


In [29]:
print ("initialize")

for m, doc in enumerate(docs):
    
    for s, sent in enumerate(doc):
        for i, word in enumerate(sent):
            v = v_d_s_n[m][s][i] # 0--T (v_d_s_n - A word can be sampled using any window covering its sentence s)
            r = r_d_s_n[m][s][i] # global or local topic
            z = z_d_s_n[m][s][i] # assigned global topic to the word

            
            if r == "gl":
                n_gl_z_w[z][word]      += 1
                n_gl_z[z]              += 1
                n_d_v_gl[m][s+v]       += 1
                n_d_gl_z[m][z]         += 1
                n_d_gl[m]              += 1

            elif r == "loc":
                n_loc_z_w[z][word]     += 1
                n_loc_z[z]             += 1
                n_d_v_loc[m][s+v]      += 1
                n_d_v_loc_z[m][s+v][z] += 1

print ("init comp.")


initialize
init comp.


In [30]:
phi_gl  = (n_gl_z_w + 1) / (n_gl_z[:, np.newaxis] + 1) # if num of topic(k) =2 then 2 vector will be generated

phi_loc = (n_loc_z_w + 1) / (n_loc_z[:, np.newaxis] + 1)


In [31]:
#v_d_s_n - sample word from sentence using any window which covers it.
#r_d_s_n - each word belong to global or local topic
#z_d_s_n - the word belong to which topic cluster of local or global

In [32]:
# for inference
for ii in range(10): # number of iteration
    for m, doc in enumerate(docs):
        for s, sent in enumerate(doc):
            for i, word in enumerate(sent):
                v = v_d_s_n[m][s][i] # 0--T
                r = r_d_s_n[m][s][i]
                z = z_d_s_n[m][s][i]

                # discount for i-th word t with topic z
                if r == "gl":
                    n_gl_z_w[z][word]      -= 1
                    n_gl_z[z]              -= 1
                    n_d_v_gl[m][s+v]       -= 1
                    n_d_gl_z[m][z]         -= 1
                    n_d_gl[m]              -= 1
                else:
                    n_loc_z_w[z][word]     -= 1
                    n_loc_z[z]             -= 1
                    n_d_v_loc[m][s+v]      -= 1
                    n_d_v_loc_z[m][s+v][z] -= 1

                # sampling topic new_z for t
                p_v_r_z = []
                label_v_r_z = []
                for v_t in range(T):
                    # for r == "gl"
                    for z_t in range(K_gl):
                        label = [v_t, "gl", z_t]
                        label_v_r_z.append(label)
                        # sampling eq as gl
                        term1 = float(n_gl_z_w[z_t][word] + beta_gl) / (n_gl_z[z_t] + W*beta_gl) # P(w|r, z)
                        term2 = float(n_d_s_v[m][s][v_t] + gamma) / (n_d_s[m][s] + T*gamma) #P(v)
                        term3 = float(n_d_v_gl[m][s+v_t] + alpha_mix_gl) / (n_d_v[m][s+v_t] + alpha_mix_gl + alpha_mix_loc) #P(r|v)
                        term4 = float(n_d_gl_z[m][z_t] + alpha_gl) / (n_d_gl[m] + K_gl*alpha_gl) #P(z|r, v)
                        score = term1 * term2 * term3 * term4
                        p_v_r_z.append(score)
                    # for r == "loc" 
                    for z_t in range(K_loc):
                        label = [v_t, "loc", z_t]
                        label_v_r_z.append(label)
                        # sampling eq as loc
                        term1 = float(n_loc_z_w[z_t][word] + beta_loc) / (n_loc_z[z_t] + W*beta_loc) # P(w|r, z)
                        term2 = float(n_d_s_v[m][s][v_t] + gamma) / (n_d_s[m][s] + T*gamma)#P(v)
                        term3 = float(n_d_v_loc[m][s+v_t] + alpha_mix_loc) / (n_d_v[m][s+v_t] + alpha_mix_gl + alpha_mix_loc)#P(r|v)
                        term4 = float(n_d_v_loc_z[m][s+v_t][z_t] + alpha_loc) / (n_d_v_loc[m][s+v_t] + K_loc*alpha_loc)#P(z|r, v)
                        score = term1 * term2 * term3 * term4
                        p_v_r_z.append(score)

                np_p_v_r_z = np.array(p_v_r_z)

                new_p_v_r_z_idx = np.random.multinomial(1, np_p_v_r_z / np_p_v_r_z.sum()).argmax()
                new_v, new_r, new_z = label_v_r_z[new_p_v_r_z_idx]

                # update the new topic assignment of word
                if new_r == "gl":
                    n_gl_z_w[new_z][word]          += 1
                    n_gl_z[new_z]                  += 1
                    n_d_v_gl[m][s+new_v]           += 1
                    n_d_gl_z[m][new_z]             += 1
                    n_d_gl[m]                      += 1
                else :
                    n_loc_z_w[new_z][word]         += 1
                    n_loc_z[new_z]                 += 1
                    n_d_v_loc[m][s+new_v]          += 1
                    n_d_v_loc_z[m][s+new_v][new_z] += 1


                v_d_s_n[m][s][i] = new_v
                r_d_s_n[m][s][i] = new_r
                z_d_s_n[m][s][i] = new_z
    
    #update other parameters
    #output_word_topic_dist
    z_gl_count = np.zeros(K_gl, dtype=int)
    z_loc_count = np.zeros(K_loc, dtype=int)
    word_gl_count = [dict() for k in range(K_gl)]
    word_loc_count = [dict() for k in range(K_loc)]
    for m, doc in enumerate(docs):
        for s, sent in enumerate(doc):
            for i, word in enumerate(sent):
#               v = mglda.v_d_s_n[m][s][i] # 0--T
                r = r_d_s_n[m][s][i]
                z = z_d_s_n[m][s][i]
                if r == "gl":
                    z_gl_count[z] += 1
                    if word in word_gl_count[z]:
                        word_gl_count[z][word]  += 1
                    else:
                        word_gl_count[z][word]   = 1
                elif r == "loc":
                    z_loc_count[z] += 1
                    if word in word_loc_count[z]:
                        word_loc_count[z][word] += 1
                    else:
                        word_loc_count[z][word]  = 1
                else:
                    print ("error3: " + str(r))
    if ii% 5 == 0:
        print(ii)
        for k in range(K_gl):
            print ("\nglobal topic: %d (%d words)" % (k, z_gl_count[k]))
            print([vocabulary[w] for w in np.argsort(-phi_gl[k])[:20]])
        print("\n******             ************                *************")

        for k in range(K_loc):
            print ("\nlocal topic: %d (%d words)" % (k, z_loc_count[k]))
            print([vocabulary[w] for w in np.argsort(-phi_loc[k])[:20]])
        print("_________________________________________________________________")


0

global topic: 0 (1006 words)
['The', 'people', 'use', 'one', 'get', 'Russian', 'know', 'Reserve', 'year', 'think', 'part', 'find', 'It', 'say', 'Armenians', 'like', 'high', 'would', 'windows', 'genocide']

global topic: 1 (1070 words)
['The', 'This', 'know', 'one', 'people', 'GPF', 'using', 'As', 'way', '180', 'like', 'State', 'Jews', 'would', 'board', 'almost', 'Armenians', 'We', 'No', 'may']

global topic: 2 (1045 words)
['The', 'would', 'think', 'Armenian', 'Russian', 'data', 'may', 'memory', '1st', 'Turkish', 'like', 'major', 'fatwa', 'Reserve', 'day', 'available', 'get', 'options', 'Information', 'used']

global topic: 3 (1096 words)
['The', 'It', 'would', 'could', 'Armenian', 'Armenians', 'This', 'think', 'used', 'After', 'No', 'might', 'As', 'That', 'like', 'car', 'tickets', 'Mac', 'application', 'left']

global topic: 4 (1071 words)
['The', 'people', 'know', 'one', 'Armenian', 'would', 'You', 'Naval', 'fact', 'many', 'Titan', 'steam', 'Armenians', 'number', 'need', 'This', '